In [0]:
# %pip install databricks-vectorsearch
# %pip install --upgrade databricks-langchain langchain-community langchain databricks-sql-connector
# dbutils.library.restartPython()
# %pip install -U mlflow
# dbutils.library.restartPython()
import mlflow
import os
from openai import OpenAI
from databricks.vector_search.client import VectorSearchClient
mlflow.openai.autolog()

In [0]:
# vsc.create_endpoint(
#     name="pizza-rag-endpoint",
#     endpoint_type="STANDARD"
# )


In [0]:
vsc = VectorSearchClient()
vsc.list_endpoints()

In [0]:
%sql
ALTER TABLE retail_analytics.pizza.gold_semantic_pizza_facts
SET TBLPROPERTIES (delta.enableChangeDataFeed = true);
ALTER TABLE retail_analytics.pizza.gold_semantic_ingredient_facts
SET TBLPROPERTIES (delta.enableChangeDataFeed = true);
ALTER TABLE retail_analytics.pizza.gold_semantic_ingredient_summary
SET TBLPROPERTIES (delta.enableChangeDataFeed = true);


In [0]:
vsc = VectorSearchClient()
vsc.create_delta_sync_index(
    endpoint_name="pizza-rag-endpoint",
    index_name="retail_analytics.pizza.gold_semantic_pizza_index",
    source_table_name="retail_analytics.pizza.gold_semantic_pizza_facts",
    primary_key="pizza_name_id",
    embedding_source_column="document",
    embedding_model_endpoint_name="databricks-bge-large-en",
    pipeline_type="TRIGGERED"
)

In [0]:
vsc.create_delta_sync_index(
    endpoint_name="pizza-rag-endpoint",
    index_name="retail_analytics.pizza.gold_semantic_ingredient_index",
    source_table_name="retail_analytics.pizza.gold_semantic_ingredient_facts",
    primary_key="pizza_name_id",
    embedding_source_column="document",
    embedding_model_endpoint_name="databricks-bge-large-en",
    pipeline_type="TRIGGERED"
)


In [0]:
vsc.create_delta_sync_index(
    endpoint_name="pizza-rag-endpoint",
    index_name="retail_analytics.pizza.gold_semantic_ingredient_summary_index",
    source_table_name="retail_analytics.pizza.gold_semantic_ingredient_summary",
    primary_key="ingredient",
    embedding_source_column="document",
    embedding_model_endpoint_name="databricks-bge-large-en",
    pipeline_type="TRIGGERED"
)

In [0]:
os.environ["OPENAI_API_KEY"] = (
    dbutils.notebook.entry_point
    .getDbutils()
    .notebook()
    .getContext()
    .apiToken()
    .get()
)

client = OpenAI(
    base_url="https://dbc-741c7540-6155.cloud.databricks.com/serving-endpoints"
)


In [0]:
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()

os.environ["DATABRICKS_HOST"] = ctx.apiUrl().get()
os.environ["DATABRICKS_TOKEN"] = ctx.apiToken().get()
api_key=os.environ["DATABRICKS_TOKEN"]
print(api_key)

In [0]:
import os, requests

def get_access_token():
    resp = requests.post(
        f"https://{os.environ['DATABRICKS_HOST']}/oidc/v1/token",
        data={
            "grant_type": "client_credentials",
            "client_id": os.environ["DATABRICKS_CLIENT_ID"],
            "client_secret": os.environ["DATABRICKS_CLIENT_SECRET"],
            "scope": "all-apis"
        }
    )
    resp.raise_for_status()
    return resp.json()["access_token"]
get_access_token()



In [0]:
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()

os.environ["DATABRICKS_HOST"] = ctx.apiUrl().get()
os.environ["DATABRICKS_TOKEN"] = ctx.apiToken().get()

class OpenAIClientFactory:
    @staticmethod
    def create():
        return OpenAI(
            base_url=f"{os.environ['DATABRICKS_HOST']}/serving-endpoints",
            api_key=os.environ["DATABRICKS_TOKEN"],
        )

In [0]:
# client = OpenAI(
#     base_url="https://dbc-741c7540-6155.cloud.databricks.com/serving-endpoints"
# )

# resp = client.chat.completions.create(
#     model="databricks-meta-llama-3-1-8b-instruct",
#     messages=[{"role": "user", "content": "Say hello in one sentence"}],
#     max_tokens=50
# )

# print(resp.choices[0].message.content)


In [0]:
from databricks_langchain import DatabricksVectorSearch
VECTOR_SEARCH_ENDPOINT = "pizza-rag-endpoint"
# VECTOR_INDEX_NAME = "retail_analytics.pizza.pizza_semantic_index"
VECTOR_INDEX_NAME = "retail_analytics.pizza.gold_semantic_pizza_index"
VECTOR_INGREDIENT_INDEX_NAME = "retail_analytics.pizza.gold_semantic_ingredient_index"
VECTOR_INGREDIENT_SUMMARY_INDEX_NAME = "retail_analytics.pizza.gold_semantic_ingredient_summary_index"
pizza_vs = DatabricksVectorSearch(
    index_name=VECTOR_INDEX_NAME,
    endpoint=VECTOR_SEARCH_ENDPOINT
)

ingredient_vs = DatabricksVectorSearch(
    index_name=VECTOR_INGREDIENT_INDEX_NAME,
    endpoint=VECTOR_SEARCH_ENDPOINT
)

ingredient_summary_vs = DatabricksVectorSearch(
    index_name=VECTOR_INGREDIENT_SUMMARY_INDEX_NAME,
    endpoint=VECTOR_SEARCH_ENDPOINT
)

In [0]:
def detect_query_intent(question: str) -> str:
    q = question.lower()

    ingredient_keywords = [
        "ingredient", "cheese", "chicken", "pepperoni",
        "olive", "onion", "tomato", "usage", "grams"
    ]

    summary_keywords = [
        "most used", "highest usage", "overall",
        "total ingredient", "dominant ingredient"
    ]

    if any(k in q for k in summary_keywords):
        return "ingredient_summary"

    if any(k in q for k in ingredient_keywords):
        return "ingredient"

    return "pizza"


In [0]:
def retrieve_context(question: str):
    intent = detect_query_intent(question)

    if intent == "ingredient_summary":
        docs = ingredient_summary_vs.similarity_search(question, k=2)

    elif intent == "ingredient":
        docs = ingredient_vs.similarity_search(question, k=3)

    else:
        docs = pizza_vs.similarity_search(question, k=3)

    return intent, docs


In [0]:
def query_pizza_rag_intent(question: str) -> str:
    intent, docs = retrieve_context(question)

    if not docs:
        return "No relevant data found for this question."

    context = "\n".join(doc.page_content for doc in docs)

    messages = [
        {
            "role": "system",
            "content": (
                "You are a senior business analyst. "
                "Answer strictly using the provided context."
            )
        },
        {
            "role": "user",
            "content": f"""
Intent: {intent}

Context:
{context}

Question:
{question}

Answer clearly with metrics where applicable.
"""
        }
    ]

    response = client.chat.completions.create(
        model="databricks-meta-llama-3-1-8b-instruct",
        messages=messages,
        temperature=0.3,
        max_tokens=512
    )

    return response.choices[0].message.content


In [0]:
question = "which ingredient was most widely used in pizza"

answer = query_pizza_rag_intent(question)

print("Question:", question)
print("\nAnswer:\n", answer)


In [0]:
from databricks.vector_search.client import VectorSearchClient

vsc = VectorSearchClient()

indexes = vsc.list_indexes(
    name="pizza-rag-endpoint"
)

display(indexes)

In [0]:
from databricks_langchain import DatabricksVectorSearch

vs = DatabricksVectorSearch(
    endpoint="pizza-rag-endpoint",
    index_name="retail_analytics.pizza.gold_semantic_ingredient_summary_index"
)

docs = vs.similarity_search(
    "which ingredient is most used",
    k=1
)

docs


In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
# w.secrets.create_scope(
#     scope="pizza-secrets"
# )
# Now you can add your secret as before
w.secrets.put_secret(
    scope="pizza-secrets",
    key="DATABRICKS_TOKEN",
    string_value=dbutils.notebook.entry_point
        .getDbutils()
        .notebook()
        .getContext()
        .apiToken()
        .get()
)

In [0]:
dbutils.secrets.get("pizza-secrets", "DATABRICKS_TOKEN")


In [0]:
import pandas as pd
from openai import OpenAI
from databricks_langchain import DatabricksVectorSearch
import os

class PizzaRAGPyFuncModel(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        self.client = OpenAI(
            base_url=f"{os.environ['DATABRICKS_HOST']}/serving-endpoints",
            api_key=os.environ["DATABRICKS_TOKEN"],
        )

        self.pizza_vs = DatabricksVectorSearch(
            index_name="retail_analytics.pizza.gold_semantic_pizza_index",
            endpoint="pizza-rag-endpoint",
        )

        self.ingredient_vs = DatabricksVectorSearch(
            index_name="retail_analytics.pizza.gold_semantic_ingredient_index",
            endpoint="pizza-rag-endpoint",
        )

        self.ingredient_summary_vs = DatabricksVectorSearch(
            index_name="retail_analytics.pizza.gold_semantic_ingredient_summary_index",
            endpoint="pizza-rag-endpoint",
        )

    def detect_query_intent(self, question: str) -> str:
        q = question.lower()

        if any(k in q for k in [
            "most used", "highest usage", "overall",
            "total ingredient", "dominant ingredient",
        ]):
            return "ingredient_summary"

        if any(k in q for k in [
            "ingredient", "cheese", "chicken", "pepperoni",
            "olive", "onion", "tomato", "grams",
        ]):
            return "ingredient"

        return "pizza"

    def retrieve_context(self, question: str):
        intent = self.detect_query_intent(question)

        if intent == "ingredient_summary":
            docs = self.ingredient_summary_vs.similarity_search(question, k=2)
        elif intent == "ingredient":
            docs = self.ingredient_vs.similarity_search(question, k=3)
        else:
            docs = self.pizza_vs.similarity_search(question, k=3)

        return intent, docs

    def predict(self, context, model_input):
        if isinstance(model_input, pd.DataFrame):
            question = model_input["query"].iloc[0]
        else:
            question = model_input["query"]

        question = str(question)

        intent, docs = self.retrieve_context(question)

        if not docs:
            return [{
                "answer": "No relevant data found.",
                "intent": intent,
                "num_documents": 0,
            }]

        context_text = "\n".join(doc.page_content for doc in docs)

        response = self.client.chat.completions.create(
            model="databricks-meta-llama-3-1-8b-instruct",
            messages=[
                {"role": "system", "content": (
                    "You are a senior business analyst. "
                    "Use ONLY the provided context. "
                    "Maintain conversational continuity."
                )},
                {"role": "user", "content": context_text + "\n\n" + question},
            ],
            temperature=0.3,
            max_tokens=512,
        )

        return [{
            "answer": response.choices[0].message.content,
            "intent": intent,
            "num_documents": len(docs),
        }]


In [0]:
import mlflow
from mlflow.models import ModelSignature
from mlflow.types import Schema, ColSpec

signature = ModelSignature(
    inputs=Schema([ColSpec("string", "query")]),
    outputs=Schema([
        ColSpec("string", "answer"),
        ColSpec("string", "intent"),
        ColSpec("long", "num_documents"),
    ]),
)

with mlflow.start_run():
    mlflow.pyfunc.log_model(
        name="pizza_rag_model",
        python_model=PizzaRAGPyFuncModel(),
        signature=signature,
        input_example={"query": "Which pizza sold the most?"},
        registered_model_name="retail_analytics.pizza.pizza_rag_models",
    )


In [0]:
from databricks_langchain import DatabricksVectorSearch

vs = DatabricksVectorSearch(
    index_name="retail_analytics.pizza.gold_semantic_ingredient_summary_index",
    endpoint="pizza-rag-endpoint"
)

vs.similarity_search("which ingredient was most widely used in pizza", k=1)


In [0]:
import requests, json

# token = dbutils.secrets.get(
#     scope="pizza-secrets",
#     key="DATABRICKS_TOKEN"
# )


url = "https://dbc-741c7540-6155.cloud.databricks.com/serving-endpoints/pizza_analytics-v16/invocations"

payload = {
    "dataframe_records": [
        {"query": "Which pizza has most order?"}
    ]
}

response = requests.post(
    url,
    headers={
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    },
    data=json.dumps(payload),
)

print(response.status_code)
print(response.json())

In [0]:
import requests
import os

url = "https://dbc-741c7540-6155.cloud.databricks.com/serving-endpoints/pizza_analytics_v10/invocations"
token = dbutils.secrets.get(
    scope="pizza-secrets",
    key="DATABRICKS_TOKEN"
)
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# payload = {
#     "inputs": [
#         {
#             "question": "Which pizza has most order?"
#         }
#     ]
# }
payload = {
    "dataframe_records": [
        {"query": "Which pizza has most order?"}
    ]
}

response = requests.post(url, headers=headers, json=payload)
response.raise_for_status()

print(response.json())
